# Demonstration of Our Adjusted TTE Estimator

This notebook demonstrates how to use Regression-based and VIM_based ADJ-SNIPE.

In [ ]:
# 1. Import Libraries
import numpy as np
import pandas as pd
import sys
sys.path.append('../src/Code_for_Experiments')
import nci_linear_setup as ncls
import nci_polynomial_setup as ncps

## 2. Generate or Load Example Data
We either simulate data or load a demo dataset. Below is a simple simulation example.

In [14]:
np.random.seed(42)
n = 5000
covariate_dim = 3
p = 0.2

# Simulate covariates
X_uncentered = np.random.randn(n, covariate_dim) 
X_c_coeff = np.ones((covariate_dim, n))+np.random.randn(covariate_dim, n)
X = X_uncentered - np.mean(X_uncentered, axis=0)

# Simulate adjacency matrix
A = ncls.erdos_renyi(n,10/n)

# Simulate treatment assignments
z = ncls.bernoulli(n,p)

# Simulate outcomes
b = 5*np.ones((covariate_dim,))
rand_wts = np.random.rand(n,3)
alpha = rand_wts[:,0].flatten()
C = ncls.simpleXWeights_old(A, X, X_c_coeff, 10, 5, rand_wts[:,1].flatten(), rand_wts[:,2].flatten())
y = ncls.linear_cov_adj(b,C,alpha,z,X)

# Input data
df = {
    'y': y,
    'z': z,
    'A': A,
    'X': X,
    'p': p
}
df

/opt/anaconda3/lib/python3.11/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


{'y': array([16.39591004,  5.86053216, 15.54424196, ...,  9.2889464 ,
         0.89849312,  3.67697727]),
 'z': array([1, 0, 0, ..., 0, 0, 1]),
 'A': <5000x5000 sparse array of type '<class 'numpy.int64'>'
 	with 55262 stored elements in Compressed Sparse Row format>,
 'X': array([[ 0.49757021, -0.14367322,  0.64596036],
        [ 1.52388592, -0.23956229, -0.23586514],
        [ 1.58006887,  0.76202582, -0.47120257],
        ...,
        [ 0.91424715,  0.2068907 ,  0.23885258],
        [-0.30306659, -0.91506094,  1.2354228 ],
        [ 1.31306405, -0.88951805, -0.15551334]]),
 'p': 0.2}

## 3. Apply the Estimator
We now apply two estimators on the dataset to estimate the total treatment effect.

In [15]:
# if degree = 1
zz = df['z']/df['p'] - (1-df['z'])/(1-df['p'])
w = df['A'].dot(zz)
n = df['X'].shape[0]

# Regression based
est_Reg = ncps.Reg_beta(n, df['y'], df['X'], w)
print(f"Degree One Reg Estimated TTE: {est_Reg:.3f}")

# VIM based
treatment_vec = df['p'] * np.ones((n,))
c_est_list = [ncls.get_c_est(df['A'], df['z'], df['y'], treatment_vec, i) for i in range(n)]
component_B = ncls.compute_component_B_deg1(df['X'], df['A'], df['p'])
component_D = ncls.compute_component_D_deg1(df['X'], df['A'], df['p'], c_est_list)
components = (component_B, component_D)
est_VIM = ncls.VIM_beta(n, df['y'], df['X'], w, components)
print(f"Degree One VIM Estimated TTE: {est_VIM:.3f}")

Degree One Reg Estimated TTE: 7.492


TypeError: compute_component_B_deg1() missing 1 required positional argument: 'p'